# Laboratorio 4 

## Importación de datos

In [1]:
import numpy
numpy.version.version

'1.23.4'

In [3]:
#importación de dataser IMD de Keras
import numpy as np
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Input, Concatenate, BatchNormalization
from keras.models import Model
import tensorflow as tf



In [7]:
# verificar si tensorflow usa GPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print(tf.config.experimental.list_physical_devices('GPU'))
print("GPUs disponibles:", len(tf.config.list_physical_devices('GPU')))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17178336854087332864
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1733715559
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9249863517159592247
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
]
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPUs disponibles: 1


In [8]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=50000)

## Pre-procesamiento

In [9]:
# Secuencie y rellene las críticas para que todas tengan una longitud uniforme.
x_train = sequence.pad_sequences(x_train, maxlen=500)
x_test = sequence.pad_sequences(x_test, maxlen=500)

In [10]:
# Preview de los datos
print(x_train.shape)
print(x_test.shape)



(25000, 500)
(25000, 500)


In [11]:
# Función para extraer características adicionales
def extract_features(sequences):
    lengths = np.array([len(seq) for seq in sequences])  # Longitud de cada crítica
    avg_word_length = np.array([np.mean([len(str(word)) for word in seq]) for seq in sequences])  # Promedio de longitud de palabras
    unique_word_ratio = np.array([len(set(seq)) / len(seq) for seq in sequences])  # Proporción de palabras únicas
    
    return np.vstack((lengths, unique_word_ratio, avg_word_length)).T

In [12]:
# Extraer características de entrenamiento y prueba
x_train_features = extract_features(x_train)
x_test_features = extract_features(x_test)

In [13]:
x_train = np.concatenate((x_train, x_train_features), axis=1)
x_test = np.concatenate((x_test, x_test_features), axis=1)

# Preview de los datos
print(x_train.shape)
print(x_test.shape)



(25000, 505)
(25000, 505)


## Modelo

In [14]:
# Crear modelo
model = Sequential()
model.add(Embedding(50000, 128, input_length=10))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 128)           6400000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 6,531,713
Trainable params: 6,531,713
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Entrenar modelo
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_test, y_test))



Epoch 1/5
 17/782 [..............................] - ETA: 1:29:57 - loss: 0.6947 - accuracy: 0.4835